<a href="https://colab.research.google.com/github/Devansh-react/PyTorch_fundamentals/blob/main/Ch_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch
import torch.nn as nn

In [28]:
class model(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.linear = nn.Linear(num_features,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self,X):
    out = self.linear(X)
    out = self.sigmoid(out)
    return out

In [29]:
features = torch.rand([10,5],dtype = torch.float32)
model = model(features.shape[1])
model(features)

tensor([[0.6843],
        [0.6137],
        [0.7045],
        [0.6477],
        [0.6266],
        [0.6036],
        [0.6395],
        [0.7282],
        [0.6333],
        [0.7232]], grad_fn=<SigmoidBackward0>)

In [30]:
#  model weights
model.linear.weight

Parameter containing:
tensor([[ 0.3444,  0.1918, -0.4213,  0.3149,  0.3739]], requires_grad=True)

In [31]:
!pip install torchinfo

In [32]:
from torchinfo import summary
summary(model,input_size = (10,5))

Layer (type:depth-idx)                   Output Shape              Param #
model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

**Model-2**

In [33]:
class model2(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.linear1 = nn.Linear(num_features,3)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(3,1)
    self.sigmoid = nn.Sigmoid()


  def forward(self,X):
    out = self.linear1(X)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.sigmoid(out)
    return out

In [34]:
feature2 = torch.rand([10,5])
model = model2(features.shape[1])
model(feature2)


tensor([[0.5802],
        [0.5795],
        [0.5806],
        [0.5738],
        [0.5773],
        [0.5714],
        [0.5780],
        [0.5757],
        [0.5806],
        [0.5757]], grad_fn=<SigmoidBackward0>)

In [35]:
model.linear1.weight , model.linear2.weight

(Parameter containing:
 tensor([[-0.2006,  0.0527,  0.0985,  0.1717, -0.3627],
         [-0.3362,  0.2704, -0.4085,  0.3432, -0.4130],
         [ 0.3760, -0.0154,  0.2335,  0.2309, -0.1478]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.2662, -0.4474, -0.2843]], requires_grad=True))

In [36]:
from torchinfo import summary
summary(model,input_size = (10,5))

Layer (type:depth-idx)                   Output Shape              Param #
model2                                   [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


**Sequencial container**

In [42]:
class model2(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.network = nn.Sequential(
    nn.Linear(num_features,3),
    nn.ReLU(),
    nn.Linear(3,1),
    nn.Sigmoid()
    )


  def forward(self,X):
    out = self.network(X)
    return out

# **Model training** on data

**MODEL pipeline**

In [40]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.drop(["Unnamed: 32","id"],axis=1,inplace=True)
X_train,X_test,Y_train,Y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.transform(Y_test)
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
Y_train_tensor = torch.from_numpy(Y_train).float()
Y_test_tensor = torch.from_numpy(Y_test).float()
X_train_tensor.shape,X_test_tensor.shape,Y_train_tensor.shape,Y_test_tensor.shape



(torch.Size([455, 30]),
 torch.Size([114, 30]),
 torch.Size([455]),
 torch.Size([114]))

In [82]:
import torch.nn as nn

class Trainig_class(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_features,1),
        nn.Sigmoid()
    )

  def forward(self,X):
    out = self.network(X)
    return out

#  paramneters
learning_rate = 0.001;
epochs = 25

In [80]:
loss_fun = nn.BCELoss()

In [83]:
model  = Trainig_class(X_train_tensor.shape[1])

#optimiser
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

for epoch in range(epochs):

  # forward-pass
  y_pred = model(X_train_tensor)
  #  loss function
  loss = loss_fun(y_pred,Y_train_tensor.view(-1,1))

  #clearing gradient
  optimizer.zero_grad()

  # backward_propogation
  loss.backward()

  # parameter_update (using in-place operations)
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.7592121362686157
Epoch: 2, Loss: 0.7569906711578369
Epoch: 3, Loss: 0.7547814846038818
Epoch: 4, Loss: 0.7525845766067505
Epoch: 5, Loss: 0.7503999471664429
Epoch: 6, Loss: 0.7482274770736694
Epoch: 7, Loss: 0.7460669279098511
Epoch: 8, Loss: 0.7439185380935669
Epoch: 9, Loss: 0.7417820692062378
Epoch: 10, Loss: 0.7396573424339294
Epoch: 11, Loss: 0.7375445365905762
Epoch: 12, Loss: 0.7354434728622437
Epoch: 13, Loss: 0.7333540916442871
Epoch: 14, Loss: 0.731276273727417
Epoch: 15, Loss: 0.7292099595069885
Epoch: 16, Loss: 0.7271551489830017
Epoch: 17, Loss: 0.7251117825508118
Epoch: 18, Loss: 0.7230797410011292
Epoch: 19, Loss: 0.7210588455200195
Epoch: 20, Loss: 0.7190492749214172
Epoch: 21, Loss: 0.7170507907867432
Epoch: 22, Loss: 0.7150633931159973
Epoch: 23, Loss: 0.7130870819091797
Epoch: 24, Loss: 0.711121678352356
Epoch: 25, Loss: 0.7091670632362366


In [84]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == Y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.49199753999710083
